<a href="https://colab.research.google.com/github/rimiiii/capsule_endoscopy_detection/blob/main/yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# google drive 마운드
# json file 불러오기
# image decoding해서 저장하기(train, test 각각)
# yolo에 맞는 label 만들기
# train test set 나누기
# yolo 모델 다운로드
# wandb 이용하기
# yaml 만들기
# 모델 train
# test
# 결과 파일 저장

In [1]:
import json
from google.colab import drive
from glob import glob
import base64
import numpy as np
import cv2
from matplotlib import pyplot as plt
import pandas as pd
import os
from tqdm.notebook import tqdm
%matplotlib inline

# google drive 마운드
drive.mount('/gdrive')

Mounted at /gdrive


In [5]:
# path 지정
base_path = '/gdrive/My Drive/data/capsule_endoscopy_detection'

train_path_list = glob(base_path+'/train/train*')
test_path_list = glob(base_path+'/test/test*')

In [7]:
# class 종류 확인하기
class_ids = pd.read_csv(base_path+'/class_id_info.csv')

# class dictionary 만들기
# 01_ulcer: 0 의 형태
categories = {}
for id in class_ids.iloc():
  categories[id['class']] = id['class_id']-1

class_ids

,class,class_id
0,01_ulcer,1
1,02_mass,2
2,04_lymph,3
3,05_bleeding,4


In [8]:
def x1x2y1y2(bbox):
  #x1,x2,y1,y2 구하기
  x1 = min(bbox[:,0])
  x2 = max(bbox[:,0])
  y1 = min(bbox[:,1])
  y2 = max(bbox[:,1])
  return x1,x2,y1,y2

def xcycwh(x1,x2,y1,y2,json_file):
  xc = (x1+x2)/2
  yc = (y1+y2)/2
  w = (x2-x1)
  h = (y2-y1)
  # img size로 나누기
  xc = np.round(xc/json_file['imageWidth'], 6)
  yc = np.round(yc/json_file['imageHeight'], 6)
  w = np.round(w/json_file['imageWidth'], 6)
  h = np.round(h/json_file['imageHeight'], 6)
  return xc,yc,w,h

In [9]:
# 함수화하기
def save_image_label(train_path_list, mode):
  #image, labels 저장할 path 만들기
  save_path = base_path+'/{}'.format(mode)
  save_image_path = save_path+'/images/'
  save_label_path = save_path+'/labels/'

  os.makedirs(save_image_path, exist_ok=True)
  os.makedirs(save_label_path, exist_ok=True)

  for train_path in tqdm(train_path_list):
    # json file 불러오기
    with open(train_path, 'r') as f:
      json_file = json.load(f)

    # image, label file path 만들기
    file_name = json_file['file_name']
    image_file_path = (save_image_path+file_name).replace('json', 'png')
    txt_file_path = (save_label_path+file_name).replace('json', 'txt')

    # image decoding해서 저장하기
    image_bytes = base64.b64decode(json_file['imageData']) # base64를 byte 단위로 데이터 저장
    img = np.frombuffer(image_bytes, dtype=np.uint8) # uint8 형태로 반환
    img = cv2.imdecode(img, cv2.IMREAD_COLOR)#image를 3차원으로 변환해줌
    cv2.imwrite(image_file_path, img) #png로 저장할땐 BGR로

    # train만 label 저장하기
    # 저장할 형태: (class, x_center, y_center, width, height)
    # 값이 0과 1 사이로 존재해야 함
    if mode == 'train':
      with open(txt_file_path, 'w') as f:
        for i in json_file['shapes']:
          bbox = np.array(i['points'])
          x1, x2, y1, y2 = x1x2y1y2(bbox)
          xc, yc, w, h  = xcycwh(x1, x2, y1, y2, json_file)
          # (class, x_center, y_center, width, height) txt 파일로 저장
          f.write(f"{categories[i['label']]}\t{xc}\t{yc}\t{w}\t{h}\n")

In [ ]:
save_image_label(train_path_list, 'train')
save_image_label(test_path_list, 'test')

  0%|          | 0/62622 [00:00<?, ?it/s]

In [ ]:
# train, validation split
from sklearn.model_selection import train_test_split

train, validation = train_test_split(train_path_list, test_size=0.33, random_state=100)

ValueError: ignored

In [ ]:
# train, validation path가 적혀있는 텍스트 파일 만들기
with open(base_path+'/train_dataset.txt', 'w') as f:
  for i in train:
    f.write(i+'\n')

with open(base_path+'/validation_dataset.txt', 'w') as f:
  for i in validation:
    f.write(i+'\n')

In [ ]:
import yaml

#yaml 파일 만들기
yaml_file = dict(
    train = base_path+'/train_dataset.txt',
    val = base_path+'/validation_dataset.txt',
    nc= 4,
    names= ['01_ulcer', '02_mass', '04_lymph', '05_bleeding'])

with open('./yolo.yaml', 'w') as f:
  yaml.dump(yaml_file, f, default_flow_style=True)

In [ ]:
# yolo 다운로드
!git clone https://github.com/ultralytics/yolov5
!pip install -qr ./yolov5/requirements.txt

In [ ]:
!pip install -q wandb
import wandb
wandb.login()

In [ ]:
import yaml

#yaml 파일 만들기
yaml_file = dict(
    path =  '/toy_train',
    train = 'images',
    val = 'iamges',
    nc= 4,
    names= ['01_ulcer', '02_mass', '04_lymph', '05_bleeding'])

with open('./yolo.yaml', 'w') as f:
  yaml.dump(yaml_file, f, default_flow_style=True)

In [ ]:
# Train YOLOv5s on COCO128 for 3 epochs

!python ./yolov5/train.py --img 576 --batch 128 --epochs 100 --data yolo.yaml --weights yolov5s.pt